<a href="https://colab.research.google.com/github/kanish777/SNLP/blob/main/Automatic_report_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
!pip install weasyprint
!pip install jinja2
import requests
from bs4 import BeautifulSoup
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from jinja2 import Template
import weasyprint

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import pandas as pd

# Create sample data
data = {
    'id': [1, 2, 3, 4, 5],
    'text': [
        "Apple is looking at buying U.K. startup for $1 billion. The company is based in London.",
        "Google has announced a new AI-based search feature. It aims to improve search accuracy.",
        "Microsoft is investing heavily in cloud computing. Their Azure platform is growing rapidly.",
        "Amazon continues to expand its logistics network. They are opening new warehouses worldwide.",
        "Facebook is working on new privacy features. They aim to protect user data more effectively."
    ]
}
# Convert to DataFrame
df = pd.DataFrame(data)

# Save to CSV
df.to_csv('sample_text.csv', index=False)

# Download the CSV file (uncomment if running in a notebook or local environment)
# from google.colab import files
# files.download('sample_text.csv')

In [ ]:
# Module 1: Data Extraction
def fetch_data(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raises HTTPError for bad responses
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return None

# Example URL with a publicly accessible HTML table
url = 'https://www.w3schools.com/html/html_tables.asp'
html_content = fetch_data(url)

if html_content:
    soup = BeautifulSoup(html_content, 'html.parser')
    table = soup.find('table')
    if table:
        data = []
        for row in table.find_all('tr'):
            cols = row.find_all(['td', 'th'])
            cols = [ele.text.strip() for ele in cols]
            if cols:  # Only append if cols is not empty
                data.append(cols)

        # Determine the maximum number of columns in the data
        max_cols = max(len(row) for row in data)

        # Fill rows with missing columns with None
        for row in data:
            while len(row) < max_cols:
                row.append(None)

        # Convert to DataFrame
        df = pd.DataFrame(data[1:], columns=data[0])
        print("Data Extraction Output:")
        print(df.head())
    else:
        print("No table found in the HTML content.")
else:
        print("Failed to retrieve data.")


Data Extraction Output:
                        Company          Contact  Country
0           Alfreds Futterkiste     Maria Anders  Germany
1    Centro comercial Moctezuma  Francisco Chang   Mexico
2                  Ernst Handel    Roland Mendel  Austria
3                Island Trading    Helen Bennett       UK
4  Laughing Bacchus Winecellars  Yoshi Tannamuri   Canada


In [93]:
#module 2: text processing and analysis
# Define text analysis functions
def analyze_text(text):
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [w for w in tokens if w.lower() not in stop_words]
    fdist = FreqDist(filtered_tokens)
    return fdist.most_common(10)  # Return top 10 most common words

def summarize_text(text):
    sentences = sent_tokenize(text)
    return '\n'.join(sentences)  # Taking all sentences as summary

# Load the CSV file
file_path = '/content/sample_text.csv'
df = pd.read_csv(file_path)

# Concatenate all text data
all_text = ' '.join(df['text'])

# Analyze and summarize the text
common_words = analyze_text(all_text)
summary = summarize_text(all_text)

# Print the results
print("\nText Analysis Output:")
print("Common Words:", common_words)
print("Summary:", summary)


Text Analysis Output:
Common Words: [('.', 10), ('new', 3), ('search', 2), ('Apple', 1), ('looking', 1), ('buying', 1), ('U.K.', 1), ('startup', 1), ('$', 1), ('1', 1)]
Summary: Apple is looking at buying U.K. startup for $1 billion.
The company is based in London.
Google has announced a new AI-based search feature.
It aims to improve search accuracy.
Microsoft is investing heavily in cloud computing.
Their Azure platform is growing rapidly.
Amazon continues to expand its logistics network.
They are opening new warehouses worldwide.
Facebook is working on new privacy features.
They aim to protect user data more effectively.


In [94]:
# Module 3: Report Generation and Formatting
def generate_report(summary, common_words):
    template = Template('''
    <html>
    <head><title>Report</title></head>
    <body>
        <h1>Automatic Report</h1>
        <h2>Summary</h2>
        <p>{{ summary }}</p>
        <h2>Common Words</h2>
        <ul>
            {% for word, frequency in common_words %}
            <li>{{ word }}: {{ frequency }}</li>
            {% endfor %}
        </ul>
    </body>
    </html>
    ''')

    html_content = template.render(summary=summary, common_words=common_words)

    # Generate PDF using weasyprint
    weasyprint.HTML(string=html_content).write_pdf("report.pdf")
    print("Report generated as 'report.pdf'")

generate_report(summary, common_words)

Report generated as 'report.pdf'


In [ ]:
# Module 1: Data Extraction
url = 'https://www.w3schools.com/html/html_tables.asp'
html_content = fetch_data(url)
soup = BeautifulSoup(html_content, 'html.parser')
table = soup.find('table')
data = []
for row in table.find_all('tr'):
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append(cols)
df = pd.DataFrame(data, columns=['Column1', 'Column2', 'Column3'])

# Module 2: Text Analysis and Summarization
sample_text = "Apple is looking at buying U.K. startup for $1 billion. The company is based in London."
common_words = analyze_text(sample_text)
summary = summarize_text(sample_text)

# Module 3: Report Generation and Formatting
generate_report(summary, common_words)

Report generated as 'report.pdf'
